# Partie 1

## Modules

In [1]:
import openpyxl
from read_input import read_input

from gurobipy import *
import math
import numpy 
import time

## Lecture des données

In [2]:
start_time = time.time()

ville="Austria" #  changer 
ressources,ressources_unavailabilities,tasks,tasks_unavailabilities = read_input(2,ville)
# V1 : "Bordeaux", "Finland", "GuineaGolf", "Italy", "Poland"
# V2 : "Austria", "Australia", "Bordeaux", "Poland", "Spain"

{'T1': <classes.Task object at 0x00000256C823F040>, 'T2': <classes.Task object at 0x00000256C823F070>, 'T3': <classes.Task object at 0x00000256C823F370>, 'T4': <classes.Task object at 0x00000256C82903A0>, 'T5': <classes.Task object at 0x00000256C82905E0>, 'T6': <classes.Task object at 0x00000256C8290550>, 'T7': <classes.Task object at 0x00000256C82904C0>, 'T8': <classes.Task object at 0x00000256C8290610>, 'T9': <classes.Task object at 0x00000256C8290640>, 'T10': <classes.Task object at 0x00000256C8290190>, 'T11': <classes.Task object at 0x00000256C82905B0>, 'T12': <classes.Task object at 0x00000256B91C0F10>, 'T13': <classes.Task object at 0x00000256B924B400>, 'T14': <classes.Task object at 0x00000256B91C4970>, 'T15': <classes.Task object at 0x00000256C81AD4C0>, 'T16': <classes.Task object at 0x00000256B9D1ED60>, 'T17': <classes.Task object at 0x00000256B7258F40>, 'T18': <classes.Task object at 0x00000256B7341AC0>, 'T19': <classes.Task object at 0x00000256C818BD60>, 'T20': <classes.Task

## Structures de données

In [3]:
def dist(lat1, long1, lat2, long2):
    """
    fonction renvoyant la distance en kms entre deux points décrits par leur latitude et longitude
    """
    if lat1==lat2 and long1==long2:
        return 0
    return(numpy.arccos(numpy.sin(2*numpy.pi/360*lat1)*numpy.sin(2*numpy.pi/360*lat2)+numpy.cos(2*numpy.pi/360*lat1)*numpy.cos(2*numpy.pi/360*lat2)*numpy.cos(2*numpy.pi/360*(long2-long1)))*6371)

#ACOS(SIN(lat1)*SIN(lat2)+COS(lat1)*COS(lat2)*COS(lon2-lon1))*6371
v=5/6

L_ressources = [j for j in ressources] # tableau des objets ressources 
L_tasks = [i for i in tasks] # tableau des objets taches
number_of_ressources = len(L_ressources)
number_of_tasks = len(L_tasks)
dict_pos = {task : (task.latitude, task.longitude) for task in tasks.values()}

dict_pos = {task : (tasks[L_tasks[task]].latitude, tasks[L_tasks[task]].longitude) for task in range(number_of_tasks)}
print(dict_pos)

#distance[a][b] donne la distance de la tâche a vers la tâche b
distances = [[dist(tasks[L_tasks[a]].latitude,tasks[L_tasks[a]].longitude, tasks[L_tasks[b]].latitude, tasks[L_tasks[b]].longitude) for b in range(number_of_tasks)] for a in range(number_of_tasks)]

#distance_m[j][i] donne la distance de la maison de j la tâche i
distances_m = [[dist(ressources[L_ressources[j]].latitude,ressources[L_ressources[j]].longitude, tasks[L_tasks[i]].latitude, tasks[L_tasks[i]].longitude) for i in range(number_of_tasks)] for j in range(number_of_ressources)]
print(distances_m)

runav_id = {}
id = 0
for j in range(len(L_ressources)):
    for u in range(len(ressources[L_ressources[j]].unavailabilities)):
        runav_id[(j,u)] = id
        id+=1
number_of_runavailabilities = id

tunav_id = {}
id = 0
for i in range(len(L_tasks)):
    for v in range(len(tasks[L_tasks[j]].unavailabilities)):
        tunav_id[(i,v)] = id
        id+=1
number_of_tunavailabilities = id


# https://gurobi.github.io/modeling-examples/technician_routing_scheduling/technician_routing_scheduling.html

{0: (48.129326, 15.754151), 1: (47.24007, 15.429835), 2: (47.803338, 15.348212), 3: (47.912735, 15.539092), 4: (47.239838, 15.231126), 5: (47.718508, 15.856215), 6: (47.691029, 15.978986), 7: (47.904305, 15.987572), 8: (48.252195, 15.15868), 9: (48.233278, 15.094242), 10: (47.878591, 14.34759), 11: (47.151374, 15.559485), 12: (48.072686, 14.499672), 13: (47.508725, 15.979283), 14: (48.182121, 15.480502), 15: (47.307187, 16.029526), 16: (48.174373, 15.768126), 17: (47.69636, 14.639145), 18: (47.465584, 15.662254), 19: (47.623501, 15.927956), 20: (47.893446, 15.138992), 21: (47.798841, 15.81416), 22: (47.430093, 15.794309), 23: (48.178337, 15.143931), 24: (47.381618, 16.094357), 25: (47.808981, 15.206235), 26: (47.947702, 15.950039), 27: (47.160069, 15.910739), 28: (47.828879, 15.253404), 29: (47.645528, 15.907399), 30: (47.399444, 15.535414)}
[[55.80303023401703, 78.57279017040275, 63.80105575756037, 53.21975500956884, 90.07207973485968, 25.22467570282596, 16.425518321648525, 25.3636421

## Variables de décision

In [4]:
### Create Model
m = Model("trs0")

M = 1440

print(number_of_ressources)
print(number_of_tasks)

### Decision variables
# Task-ressource assignment
x = {(t,r) : m.addVar(vtype = GRB.BINARY, name = f'x_{t}_{r}') for r in range(number_of_ressources) for t in range(number_of_tasks+number_of_runavailabilities)}

# Time-task assignment
s = {t : m.addVar(vtype = GRB.INTEGER, lb = 0, ub = 1440, name = f's_{t}') for t in range(number_of_tasks+number_of_runavailabilities)}
# Les s_t valent entre 0 et 1440 car les contraintes de temps (entre 8h et 18h) sont contenues dans les jeux de données.

# Est-ce que le travailleur se déplace entre les points a et b ?
y = {(r,a,b) : m.addVar(vtype = GRB.BINARY, name = f'y_{r}_{a}_{b}') for r in range(number_of_ressources) for a in range(number_of_tasks+number_of_runavailabilities) for b in range(number_of_tasks+number_of_runavailabilities)}# if a != b}
# Technician cannot leave or return to a depot that is not its base

# Is the worker used ?
u = {r: m.addVar(vtype = GRB.BINARY, name = f'u_{r}') for r in range(number_of_ressources)}

# starting task of worker
d = {(r,t) : m.addVar(vtype = GRB.BINARY, name = f'd_{r}_{t}') for r in range(number_of_ressources) for t in range(number_of_tasks+number_of_runavailabilities)}
# ending task of worker
f = {(r,t) : m.addVar(vtype = GRB.BINARY, name = f'f_{r}_{t}') for r in range(number_of_ressources) for t in range(number_of_tasks+number_of_runavailabilities)}

#variables de linéarisation
#H = {(r,a,b) : m.addVar(vtype = GRB.BINARY,lb = 0, ub = 1440, name = f'h_{r}_{a}_{b}') for r in range(number_of_ressources) for a in range(number_of_tasks) for b in range(number_of_tasks)}# if a != b}
#P = {(r,a,b) : m.addVar(vtype = GRB.BINARY,lb = 0, ub = 1440, name = f'p_{r}_{a}_{b}') for r in range(number_of_ressources) for a in range(number_of_tasks) for b in range(number_of_tasks)}# if a != b}

a = {v : m.addVar(vtype = GRB.BINARY, name = f'a_{v}') for v in range(number_of_tunavailabilities)}
z = {(r,a,b) : m.addVar(vtype = GRB.BINARY, name = f'y_{r}_{a}_{b}') for r in range(number_of_ressources) for a in range(number_of_tasks+number_of_runavailabilities) for b in range(number_of_tasks+number_of_runavailabilities)}# if a != b}


Set parameter Username
Academic license - for non-commercial use only - expires 2023-02-06
5
31


## Contraintes

In [5]:
# une personne est obligée de se déplacer (pas d'arrêtes de bouclage sur un point du graphe)
C4_2 = [m.addConstr(y[(r,a,a)] == 0) for r in range(number_of_ressources) for a in range(number_of_tasks+number_of_runavailabilities)]

In [6]:
nbRpT = [LinExpr() for t in range(number_of_tasks)]
for t in range(number_of_tasks):
  for r in  range(number_of_ressources):
    nbRpT[t]+=x[(t,r)]

C0 = [m.addConstr(x[(runav_id[(r,u)]+number_of_tasks,r)] == 1) for (r,u) in runav_id]
# One ressource per task
C1 = [m.addConstr(nbRpT[t] <= 1) for t in range(number_of_tasks)]

KeyError: (1, 31)

In [ ]:
nbRpTs = [LinExpr() for t in range(number_of_tasks)]
for t in range(number_of_tasks):
  for r in  range(number_of_ressources):
     nbRpTs[t]+=x[(t,r)]*(ressources[L_ressources[r]].skill!=tasks[L_tasks[r]].skill)

 # Skilled technicians
C2 = [m.addConstr(nbRpTs[t] == 0) for t in range(number_of_tasks)]

In [ ]:
nbRpTl =[ LinExpr() for t in range(number_of_tasks)]
for t in range(number_of_tasks):
  for r in  range(number_of_ressources):
    nbRpTl[t]+=x[(t,r)]*(ressources[L_ressources[r]].level-tasks[L_tasks[t]].level)

# Leveled technicians
C3 = [m.addConstr(nbRpTl[t] >= 0) for t in range(number_of_tasks)]

In [ ]:
# si une personne est affectée, elle arrive jusqu'à la tâche
C4 = {(i,j) : m.addConstr(quicksum(y[(j,a,i)] for a in range(number_of_tasks+number_of_runavailabilities)) + d[(j,i)] == x[(i,j)], name = f'arrive{i}{j}') for j in range(number_of_ressources) for i in range(number_of_tasks+number_of_runavailabilities)}

# une personne doit partir après avoir exécuté sa tâche
C5 = {(i,j) : m.addConstr(quicksum(y[(j,i,b)] for b in range(number_of_tasks+number_of_runavailabilities)) + f[(j,i)] == x[(i,j)], name = f'part{i}{j}') for j in range(number_of_ressources) for i in range(number_of_tasks+number_of_runavailabilities)}

In [ ]:
# non overlapping tasks
print(distances[0][1]/v)
print(tasks[L_tasks[0]].duration)
C6 = [m.addConstr(s[b] >= s[a] + tasks[L_tasks[a]].duration + distances[a][b]/v -M*(1-quicksum(y[(j,a,b)] for j in range(number_of_ressources)))) for a in range(number_of_tasks+number_of_runavailabilities) for b in range(number_of_tasks+number_of_runavailabilities)]

In [ ]:
# la premiere tache commence apres le début de la journée
#j,i=0,3
#print(ressources[L_ressources[j]].workingStart -M*(1-x[(i,j)].x) + distances_m[j][i]/v)
#M=10E2
C10 = [m.addConstr(s[i] >= ressources[L_ressources[j]].workingStart -M*(1-x[(i,j)]) + distances_m[j][i]/v) for i in range(number_of_tasks+number_of_runavailabilities) for j in range(number_of_ressources)]

In [ ]:
# la derniere tache finit avant la fin de la journee
C11 = [m.addConstr(ressources[L_ressources[j]].workingEnd >= s[i]+tasks[L_tasks[i]].duration + distances_m[j][i]/v -M*(1-x[(i,j)])) for i in range(number_of_tasks+number_of_runavailabilities) for j in range(number_of_ressources)]

In [ ]:
# une personne doit partir de chez elle si elle travaille ce jour la
C7 = [m.addConstr(quicksum(d[(j,a)] for a in range(number_of_tasks+number_of_runavailabilities)) == u[j]) for j in range(number_of_ressources)]
# une personne doit revenir chez elle si elle travaille ce jour la
C8 = [m.addConstr(quicksum(f[(j,b)] for b in range(number_of_tasks+number_of_runavailabilities)) == u[j]) for j in range(number_of_ressources)]

# contraintes pour lier x et u
C9 = [m.addConstr(quicksum(x[(i,j)] for i in range(number_of_tasks+number_of_runavailabilities)) <= u[j]*M) for j in range(number_of_ressources)]


In [ ]:
# contrainte évitant d'avoir des cycles dans les graphes (permet que les tâches soient reliées entre elles)
C19 = [m.addConstr(y[(j,a,b)] + y[(j,b,a)] <=1) for a in range(number_of_tasks) for b in range(number_of_tasks) for j in range(number_of_ressources)]
C20 = [m.addConstr(d[(j,a)] + f[(j,a)] <=1) for a in range(number_of_tasks) for j in range(number_of_ressources)]

In [ ]:
# contraintes d'horaires d'ouverture et de fermeture des tâches
C12 = [m.addConstrs(s[i] >= tasks[L_tasks[i]].openingTime  -M*(1-quicksum(x[(i,j)] for j in range (number_of_ressources))) for i in range(number_of_tasks))]
C13 = [m.addConstrs(tasks[L_tasks[i]].closingTime >= s[i] + tasks[L_tasks[i]].duration - M*(1-quicksum(x[(i,j)] for j in range (number_of_ressources)))for i in range(number_of_tasks))]

In [ ]:
C14 = [[[m.addConstr(y[(r,a,b)]+s[a]+60+tasks[L_tasks[a]].duration <= tasks[L_tasks[b]].duration + z[(a,b)]) for r in range(number_of_ressources)] for b in range (number_of_tasks+number_of_runavailabilities)] for a in range(number_of_tasks+number_of_runavailabilities)]
C15 = [m.addConstrs(s[b] >= 13*60 - M*(1-quicksum(z[(r,a,b)] for r in range(number_of_ressources))) for a in range(number_of_tasks+number_of_runavailabilities) for b in range(number_of_tasks+number_of_runavailabilities))] 
C16 = [m.addConstrs(s[a] <= 13*60 + M*(1-quicksum(z[(r,a,b)] for r in range(number_of_ressources))) for a in range(number_of_tasks+number_of_runavailabilities) for b in range(number_of_tasks+number_of_runavailabilities))]
C17 = [m.addConstr(s[t] -M*(1-quicksum(x[(i,j)] for j in range (number_of_ressources))) + tasks[L_tasks[i]].duration <= tasks[L_tasks[t]].unavailibilities[R].unavailabilityStart for (t,R) in runav_id for t in range(number_of_tasks))]
C18 = [m.addConstr(s[t] -M*(1-quicksum(x[(i,j)] for j in range (number_of_ressources))) + M*(1-a[(t,R)]) >= tasks[L_tasks[t]].unavailibilities[R].unavailabilityEnd for (t,R) in runav_id for t in range(number_of_tasks))]

## Fonction objectif et résolution

In [ ]:
epsilon = 0.0001

couts=LinExpr()
for j in range(number_of_ressources):
  for a in range(number_of_tasks+number_of_runavailabilities):
    for b in range(number_of_tasks+number_of_runavailabilities):
      couts+=y[(j,a,b)]*distances[a][b]/v
    couts+=d[(j,a)]*distances_m[j][a]/v
    couts+=f[(j,a)]*distances_m[j][a]/v

duree=LinExpr()
for r in range(number_of_ressources):
  for t in range(number_of_tasks):
    duree+=x[(r,t)]*tasks[L_tasks[t]].duration

# minimiser-(la durée totale des tâches exécutées)
obj1 = -duree
# minimiser le coût opérationnel
obj2 = couts

solutions_efficaces = list()

# -- Ajout de la fonction objectif --
m.setObjective(obj1, GRB.MINIMIZE)

# -- Choix d'un paramétrage d'affichage minimaliste --
m.params.outputflag = 0 # mode muet
m.update()
m.display()
m.optimize()


it = 0
while m.status != GRB.INFEASIBLE:
    sol = (round(m.objVal,2), round(obj2.getValue()*100,2))
    print("itération {} solution {}".format(it, sol))
    solutions_efficaces.append(sol)
    
    
    # - 
    it += 1
    # -- Ajout de l'epsilon constraint
    m.addConstr(obj2 <= obj2.getValue() - epsilon, name=f'epsilon_constraint_{it}')
    # -- Mise à jour du modèle  --
    m.update()
    # -- Résolution --
    m.optimize()

In [ ]:
# -- Affichage de la solution --

t=open("Solution"+ville+"V2ByV4.txt",'w'.format(ville))
res="taskId;performed;employeeName;startTime;\n"
for j in range(number_of_tasks):
  p=False
  for i in range(number_of_ressources):
    if round(x[(j,i)].x):
      res+="{};{};{};{};\n".format(L_tasks[j],1,L_ressources[i],int(s[j].x))
      p=True
  if not p:
    res+="{};;;;\n".format(L_tasks[j])
print(res)
t.write(res)
t.close()

print("###########################")

for a in range(number_of_tasks):
  for b in range(number_of_tasks):
    j=0
    if  round(y[(j,a,b)].x)==1:
      print("j{} - a{} - b{} - y{} // ".format(j, a,b,y[(j,a,b)].x), end="")

print('\n') 
print("###########################")

for a in range(number_of_tasks):
  j=0
  if  round(d[(j,a)].x)==1:
    print("j{} - a{} - d{} // ".format(j, a,d[(j,a)].x), end="")


print('\n\n')
for a in range(number_of_tasks):
  j=0
  if  round(f[(j,a)].x)==1:
    print("j{} - a{} - d{} // ".format(j, a,f[(j,a)].x), end="")

interval = time.time() - start_time

In [ ]:
print("temps de calcul : ", interval)
print("valeur de la fonction objectif : ", m.ObjVal)

## Vérification de la solution

In [ ]:
# affichage de la carte
import folium

lat_tot,long_tot=0,0
for a in range(number_of_tasks):
  lat_tot+=tasks[L_tasks[a]].latitude
  long_tot+=tasks[L_tasks[a]].longitude
mid_latitude = lat_tot/number_of_tasks
mid_longitude = long_tot/number_of_tasks
carte=folium.Map(location=[mid_latitude, mid_longitude],zoom_start=10)
for a in range(number_of_tasks):
  folium.Marker([tasks[L_tasks[a]].latitude, tasks[L_tasks[a]].longitude],icon=folium.Icon(color='black')).add_to(carte)
list_colors = ['darkgreen', 'blue', 'red', 'green', 'darkpurple', 'lightred', 'purple', 'pink', 'orange', 'darkblue', 'gray', 'darkred']
number_of_colors = len(list_colors)
print(number_of_colors)
counter = 0
for r in range(number_of_ressources):
  color_r = list_colors[counter % number_of_colors]
  folium.Marker([ressources[L_ressources[r]].latitude,ressources[L_ressources[r]].longitude],icon=folium.Icon(color=color_r)).add_to(carte)
  for a in range(number_of_tasks):
    for b in range(number_of_tasks):
      if round(y[(r,a,b)].x) == 1:
        line_ab = [(tasks[L_tasks[a]].latitude, tasks[L_tasks[a]].longitude),(tasks[L_tasks[b]].latitude, tasks[L_tasks[b]].longitude)]
        folium.PolyLine(line_ab, color=color_r, weight=2.5, opacity=0.8).add_to(carte)
    if round(d[(r,a)].x) == 1:
      line_home_t = [(ressources[L_ressources[r]].latitude,ressources[L_ressources[r]].longitude), (tasks[L_tasks[a]].latitude, tasks[L_tasks[a]].longitude)]
      folium.PolyLine(line_home_t, color=color_r, weight=2.5, opacity=0.8).add_to(carte)
    if round(f[(r,a)].x) == 1:
      line_t_home = [(tasks[L_tasks[a]].latitude, tasks[L_tasks[a]].longitude), (ressources[L_ressources[r]].latitude,ressources[L_ressources[r]].longitude)]
      folium.PolyLine(line_t_home, color=color_r, weight=2.5, opacity=0.8).add_to(carte)
  counter+=1

carte.save('Cartographie/Cartev1.html')

carte

In [ ]:
import plotly.express as px
import pandas as pd

# plotting the working tasks
liste=[]
for i in range(number_of_ressources):
    for j in range(number_of_tasks):
        if round(x[(j,i)].x):
            start=int(s[j].x)
            end=(int(s[j].x)+tasks[L_tasks[j]].duration)
            liste.append(dict(Task=L_ressources[i], Start=start, Finish=end, Resource="executing task",J="tâche "+str(j)))

# plotting the transports
for a in range(number_of_tasks):
    for b in range(number_of_tasks):
        for j in range(number_of_ressources):
            if round(y[(j,a,b)].x):
                start=(int(s[a].x)+tasks[L_tasks[a]].duration)
                end=(start+distances[a][b]/v)
                liste.append(dict(Task=L_ressources[j], Start=start, Finish=end, Resource="moving",J="trajet "+str(a)+"-"+str(b)))

# plotting the first transport
for a in range(number_of_tasks):
    for j in range(number_of_ressources):
        if round(d[(j,a)].x):
            start=(s[a].x-distances_m[j][a]/v)
            end=s[a].x
            liste.append(dict(Task=L_ressources[j], Start=start, Finish=end, Resource="depart",J="trajet maison -"+str(a)))

# plotting the last transport
for a in range(number_of_tasks):
    for j in range(number_of_ressources):
        if round(f[(j,a)].x):
            start=(s[a].x + tasks[L_tasks[i]].duration)
            end=(start+ distances_m[j][a]/v)
            liste.append(dict(Task=L_ressources[j], Start=start, Finish=end, Resource="depart",J=str(a)+"- trajet maison"))

df = pd.DataFrame(liste)
df['delta'] = df['Finish'] - df['Start']

fig = px.timeline(df, x_start="Start", x_end="Finish", y="Task",text="J")

fig.layout.xaxis.type = 'linear'
fig.data[0].x = df.delta.tolist()
fig.show()

In [ ]:
# check que les techniciens ont le bon niveau et les bonnes compétences pour les tâches exécutées
a = True
for i in range(number_of_ressources):
    for j in range(number_of_tasks):
        if round(x[(j,i)].x):
            # check que le skill est bon
            if ressources[L_ressources[i]].skill!=tasks[L_tasks[i]].skill:
                print("fail r,t ",i,j)
                a = False
            # check que le level est bon
            if ressources[L_ressources[i]].level < tasks[L_tasks[j]].level:
                print("fail r,t ",i,j)
                a = False
if a :
    print("Les techniciens ont bien les bonnes compétences et le bon niveau pour toutes les tâches exécutées")
            

In [ ]:
# check que les contraintes d'ouverture et de fermeture des tâches sont respectées (à réécrire pour la v2)
a = True
for i in range(number_of_ressources):
    for j in range(number_of_tasks):
        if round(x[(j,i)].x):
            if s[i].x < tasks[L_tasks[i]].openingTime:
                print("fail r,t ",i,j)
                a = False
            if tasks[L_tasks[i]].closingTime < s[i].x + tasks[L_tasks[i]].duration :
                print("fail r,t ",i,j)
                a = False

            
            
if a :
    print("Les contraintes d'ouverture et de fermeture des tâches sont respectées")

In [ ]:
"""
coder convexite

check que les tasks et ressources unavailabilities sont respectées

coder EDT pause dej

lieu propre deb et fin de journee
"""